In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Problem 3: Metro Interstate Traffic Volume

In [2]:
# load data
url = 'https://raw.githubusercontent.com/um-perez-alvaro/Data-Science-Practice/master/Data/metro.csv'
data = pd.read_csv(url,index_col='date_time',parse_dates=True)
data.head()

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,weather_description,traffic_volume
date_time,,,,,,,,
2016-01-01 00:00:00,New Years Day,265.94,0.0,0.0,90,Haze,haze,1513
2016-01-01 00:00:00,New Years Day,265.94,0.0,0.0,90,Snow,light snow,1513
2016-01-01 01:00:00,None,266.00,0.0,0.0,90,Snow,light snow,1550
2016-01-01 03:00:00,None,266.01,0.0,0.0,90,Snow,light snow,719
2016-01-01 04:00:00,None,264.80,0.0,0.0,90,Clouds,overcast clouds,533


This dataset contains information about the hourly traffic volume on the West-bound lane of I-94 (**MN DoT ATR station 301**), roughly midway between Minneapolis and St Paul, MN. The dataset includes hourly weather and temperature reports from 2016 to 2018.

<table><tr>
<td> <img src="i-94a.png" alt="Drawing" style="width: 500px;"/> </td>
<td> <img src="i-94b.png" alt="Drawing" style="width: 500px;"/> </td>
</tr></table>

| Column | Description |
| :-- | --: | 
| `date_time` (index) | Hour of the data collected in local CST time|
| `holiday` | US National holidays plus regional holiday (Minnesota State Fair) | 
| `temp` | Average temp (in kelvin) |
| `rain_1h` | Amount in mm of rain that occurred in the hour |
| `snow_1h` | Amount in mm of snow that occurred in the hour |
| `clouds_all` | Percentage of cloud cover |
| `weather_main` | Short textual description of the current weather |
| `weather_description` | Longer textual description of the current weather |
| `traffic_volume` | Hourly I-94 ATR 301 reported westbound traffic volume |

In [3]:
data

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,weather_description,traffic_volume
date_time,,,,,,,,
2016-01-01 00:00:00,New Years Day,265.94,0.0,0.0,90,Haze,haze,1513
2016-01-01 00:00:00,New Years Day,265.94,0.0,0.0,90,Snow,light snow,1513
2016-01-01 01:00:00,None,266.00,0.0,0.0,90,Snow,light snow,1550
2016-01-01 03:00:00,None,266.01,0.0,0.0,90,Snow,light snow,719
2016-01-01 04:00:00,None,264.80,0.0,0.0,90,Clouds,overcast clouds,533
...,...,...,...,...,...,...,...,...
2018-09-30 19:00:00,None,283.45,0.0,0.0,75,Clouds,broken clouds,3543
2018-09-30 20:00:00,None,282.76,0.0,0.0,90,Clouds,overcast clouds,2781
2018-09-30 21:00:00,None,282.73,0.0,0.0,90,Thunderstorm,proximity thunderstorm,2159


In [63]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Custom transformer for extracting time-based features
class TimeFeaturesExtractor(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        # Copy the input dataframe to avoid changing it outside the scope of the transformer
        X_transformed = X.copy()
        X_transformed['hour'] = X_transformed.index.hour
        X_transformed['day_of_week'] = X_transformed.index.dayofweek
        X_transformed['month'] = X_transformed.index.month
        return X_transformed
    

# Reload the dataset
traffic_data = data

# Remove duplicate rows
traffic_data_cleaned = traffic_data.drop_duplicates()

# Separate the features and the target variable
X = traffic_data_cleaned.drop(columns=['traffic_volume'], axis=1)
y = traffic_data_cleaned['traffic_volume']

# Split the data into training (80%) and testing (20%) sets without shuffling
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

# Define the column transformer to handle the one-hot encoding
categorical_features = ['holiday', 'weather_main', 'weather_description']
preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(drop='first'), categorical_features)
    ],
    remainder='passthrough'  # passthrough features not listed in transformers
)

# Adjusting the column transformer to avoid sparse output
preprocessor.set_params(sparse_threshold=0)

# Create the pipeline
pipeline = Pipeline(steps=[
    ('time_features', TimeFeaturesExtractor()),  # Time-based features extraction
    ('preprocessor', preprocessor),               # One-hot encoding
    ('scaler', StandardScaler()),                 # Feature scaling
    ('regressor', RandomForestRegressor(random_state=42, n_jobs=-1))  # Model
])

# Apply the pipeline
pipeline.fit(X_train, y_train)

# Predict and evaluate
y_pred_pipeline = pipeline.predict(X_test)
rmse_pipeline = mean_squared_error(y_test, y_pred_pipeline, squared=False)
print('RMSE for Random Forest with Pipeline: {:.4f}'.format(rmse_pipeline))
pipeline

RMSE for Random Forest with Pipeline: 500.9921


Pipeline(steps=[('time_features', TimeFeaturesExtractor()),
                ('preprocessor',
                 ColumnTransformer(remainder='passthrough', sparse_threshold=0,
                                   transformers=[('onehot',
                                                  OneHotEncoder(drop='first'),
                                                  ['holiday', 'weather_main',
                                                   'weather_description'])])),
                ('scaler', StandardScaler()),
                ('regressor',
                 RandomForestRegressor(n_jobs=-1, random_state=42))])

In [38]:
# get the trainsformed data form the pipeline
X_train_transformed = pipeline.named_steps['time_features'].transform(X_train)
X_train_transformed = pipeline.named_steps['preprocessor'].transform(X_train_transformed)
X_train_final = pipeline.named_steps['scaler'].transform(X_train_transformed)
X_test_transformed = pipeline.named_steps['time_features'].transform(X_test)
X_test_transformed = pipeline.named_steps['preprocessor'].transform(X_test_transformed)
X_test_final = pipeline.named_steps['scaler'].transform(X_test_transformed)

In [42]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid for GridSearch
param_grid = {
    'n_estimators': [100, 200],  # The number of trees in the forest
    'max_depth': [None, 2, 4], # The maximum depth of the tree
    'min_samples_leaf': [1, 2, 4, 8, 12, 14, 16, 18]         # The minimum number of samples required to be at a leaf node
}

# Create a GridSearchCV object
grid_search = GridSearchCV(estimator=RandomForestRegressor(random_state=42, n_jobs=-1), param_grid=param_grid, 
                           cv=3,  # Number of cross-validation folds
                           scoring='neg_mean_squared_error',  # You can choose another if you wish
                           verbose=2,  # Controls the verbosity: the higher, the more messages
                           n_jobs=-1)  # Number of jobs to run in parallel

# Fit the GridSearchCV object to the training data
grid_search.fit(X_train_final, y_train)

# Get the best combination of parameters
best_params = grid_search.best_params_
print('Best parameters found:', best_params)

# Use the best estimator to make predictions
y_pred_best = grid_search.best_estimator_.predict(X_test_final)

# Calculate RMSE with the best estimator
rmse_best = mean_squared_error(y_test, y_pred_best, squared=False)
print('RMSE for Random Forest with Best Parameters: {:.4f}'.format(rmse_best))

Fitting 3 folds for each of 48 candidates, totalling 144 fits
Best parameters found: {'max_depth': None, 'min_samples_leaf': 16, 'n_estimators': 200}
RMSE for Random Forest with Best Parameters: 475.3042


In [53]:
X_data = pd.DataFrame(X_test_final)
# add column names to X_data
X_data.columns = pipeline.named_steps['preprocessor'].get_feature_names_out()

Your **goal** is to **train** a regression pipeline that predicts the hourly traffic volume, **tune** the pipeline hyperparameters, and **test** its performance.